### STEP 0. IMPORTING AND SETTING UP USER AGENT

In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

#we need to use a user agent here, because metacritics does not accept bots
#we are using a tested go-to user agent here, it can be found online
#read more about user agents here:
#https://www.scrapehero.com/how-to-fake-and-rotate-user-agents-using-python-3/
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

# WEBSITE: www.metacritic.com

### LINK: https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc

### STEP 1. EXTRACTING MOVIE NAMES

In [2]:
url = "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc"
page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.content, "html.parser")

container = soup.find_all('td', class_ = 'clamp-summary-wrap')
#print(type(container))

#then lets find out how many movies we got
len(container)

movie_names = []
movies = container[:50] #here we get the top 50 movies we want
for movie in movies:
    name = movie.find('h3').text
    movie_names.append(name)
print(movie_names)

['Citizen Kane', 'The Godfather', 'Rear Window', 'Casablanca', 'Boyhood', 'Three Colors: Red', 'Vertigo', 'Notorious', "Singin' in the Rain", 'City Lights', 'Moonlight', 'Intolerance', 'Pinocchio', 'Touch of Evil', 'The Treasure of the Sierra Madre', "Pan's Labyrinth", 'Some Like It Hot', 'North by Northwest', 'Hoop Dreams', 'Rashomon', 'All About Eve', 'Jules and Jim', 'The Wild Bunch', 'My Left Foot', 'The Third Man', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb', 'Nomadland', 'Gone with the Wind', '4 Months, 3 Weeks and 2 Days', 'Psycho', 'Battleship Potemkin', 'A Streetcar Named Desire', 'American Graffiti', 'Dumbo', 'Roma', 'Ran', 'The Shop Around the Corner', '12 Angry Men', 'Manchester by the Sea', "Rosemary's Baby", 'The Maltese Falcon', '12 Years a Slave', 'Killer of Sheep', 'Rocks', 'Nashville', 'Ratatouille', 'Parasite', "Don't Look Now", 'The Grapes of Wrath', 'Children of Paradise (1945)']


### STEP 2. EXTRACTING GENRES

In [3]:
#genres
#now we need to click into each movie name in order to see detailed information
#so we need to extract all the urls
movie_links = []
movies = container[:50]
for movie in movies:
    tag = movie.find('a', class_ = 'title')
    link = tag.get('href', None)
    movie_links.append('https://www.metacritic.com'+link)
#print(movie_links)

import re
genre_list = []
c=0
for i in movie_links:
    page = requests.get(i, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    tags = soup.find_all('div', class_ = 'genres')
    for each in tags:
        temp = each.contents[3].text
        span = temp.strip().rstrip()
        #span = span.rstrip()
        span = span.replace(" ", "");
        genre_list.append(span.split(","))
print(genre_list)

[['Drama', 'Mystery'], ['Drama', 'Thriller', 'Crime'], ['Mystery', 'Thriller'], ['Drama', 'Romance', 'War'], ['Drama'], ['Drama', 'Mystery', 'Romance'], ['Mystery', 'Thriller', 'Romance'], ['Drama', 'Thriller', 'Romance', 'Film-Noir'], ['Comedy', 'Romance', 'Musical'], ['Drama', 'Comedy', 'Romance'], ['Drama'], ['Drama', 'History'], ['Fantasy', 'Comedy', 'Animation', 'Family', 'Musical'], ['Drama', 'Thriller', 'Crime', 'Film-Noir'], ['Adventure', 'Drama', 'Western'], ['Drama', 'Mystery', 'Thriller', 'Fantasy', 'War'], ['Comedy', 'Romance'], ['Action', 'Adventure', 'Mystery', 'Thriller'], ['Drama', 'Sport', 'Documentary'], ['Drama', 'Mystery', 'Crime'], ['Drama'], ['Drama', 'Romance'], ['Action', 'Adventure', 'Western'], ['Biography', 'Drama'], ['Mystery', 'Thriller', 'Film-Noir'], ['Drama', 'Thriller'], ['Drama'], ['Drama', 'History', 'Romance', 'War'], ['Drama'], ['Mystery', 'Thriller', 'Horror'], ['Drama', 'History', 'Thriller', 'War'], ['Drama'], ['Drama', 'Comedy'], ['Drama', 'Anim

### STEP 3. CREATING A DICTIONARY

In [4]:
movies_genre_meta ={}
keys = movie_names
count = 0
for i in keys:
    movies_genre_meta[i] = genre_list[count]
    count = count + 1
    
print(movies_genre_meta)

{'Citizen Kane': ['Drama', 'Mystery'], 'The Godfather': ['Drama', 'Thriller', 'Crime'], 'Rear Window': ['Mystery', 'Thriller'], 'Casablanca': ['Drama', 'Romance', 'War'], 'Boyhood': ['Drama'], 'Three Colors: Red': ['Drama', 'Mystery', 'Romance'], 'Vertigo': ['Mystery', 'Thriller', 'Romance'], 'Notorious': ['Drama', 'Thriller', 'Romance', 'Film-Noir'], "Singin' in the Rain": ['Comedy', 'Romance', 'Musical'], 'City Lights': ['Drama', 'Comedy', 'Romance'], 'Moonlight': ['Drama'], 'Intolerance': ['Drama', 'History'], 'Pinocchio': ['Fantasy', 'Comedy', 'Animation', 'Family', 'Musical'], 'Touch of Evil': ['Drama', 'Thriller', 'Crime', 'Film-Noir'], 'The Treasure of the Sierra Madre': ['Adventure', 'Drama', 'Western'], "Pan's Labyrinth": ['Drama', 'Mystery', 'Thriller', 'Fantasy', 'War'], 'Some Like It Hot': ['Comedy', 'Romance'], 'North by Northwest': ['Action', 'Adventure', 'Mystery', 'Thriller'], 'Hoop Dreams': ['Drama', 'Sport', 'Documentary'], 'Rashomon': ['Drama', 'Mystery', 'Crime'], '

# WEBSITE: www.imdb.com

#### LINK: https://www.imdb.com/chart/top/

### STEP 1. EXTRACTING MOVIE NAMES 

In [5]:
url = "https://www.imdb.com/chart/top/"
page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.content, "html.parser")

container = soup.find_all('td', class_ = 'titleColumn')

movie_names = []
movies = container[:50] #here we get the top 50 movies we want
for movie in movies:
    name = movie.find('a').text
    movie_names.append(name)
print(movie_names)# WEBSITE: www.imdb.com

#### LINK: https://www.imdb.com/chart/top/

### STEP 1. EXTRACTING MOVIE NAMES 

['The Shawshank Redemption', 'The Godfather', 'The Godfather: Part II', 'The Dark Knight', '12 Angry Men', "Schindler's List", 'The Lord of the Rings: The Return of the King', 'Pulp Fiction', 'The Good, the Bad and the Ugly', 'The Lord of the Rings: The Fellowship of the Ring', 'Fight Club', 'Forrest Gump', 'Inception', 'The Lord of the Rings: The Two Towers', 'Star Wars: Episode V - The Empire Strikes Back', 'The Matrix', 'Goodfellas', "One Flew Over the Cuckoo's Nest", 'Seven Samurai', 'Se7en', 'Life Is Beautiful', 'City of God', 'The Silence of the Lambs', "It's a Wonderful Life", 'Star Wars: Episode IV - A New Hope', 'Saving Private Ryan', 'Spirited Away', 'The Green Mile', 'Interstellar', 'Parasite', 'Léon: The Professional', 'The Usual Suspects', 'Hara-Kiri', 'The Lion King', 'The Pianist', 'Back to the Future', 'Terminator 2: Judgment Day', 'American History X', 'Modern Times', 'Psycho', 'Gladiator', 'City Lights', 'The Departed', 'The Intouchables', 'Whiplash', 'The Prestige', 

### STEP 2. EXTRACTING GENRES

In [6]:
movie_links = []
movies = container[:50]
for movie in movies:
    tag = movie.find('a')
    link = tag.get('href', None)
    movie_links.append(link)
print("Total links", len(movie_links))

for i in range(len(movie_links)):
    movie_links[i] = 'https://www.imdb.com'+ movie_links[i]
    
#movie_links

import re
genre_list=[]
for i in movie_links:
    movie=[]
    page = requests.get(i, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    tags = soup.find("h4", text="Genres:").parent
    anc= tags.select("a")
    for each in anc:
        temp = each.text
        temp = temp.replace(" ","")
        movie.append(temp)
    genre_list.append(movie[:])
    movie.clear()
print(genre_list)

Total links 50
[['Drama'], ['Crime', 'Drama'], ['Crime', 'Drama'], ['Action', 'Crime', 'Drama', 'Thriller'], ['Crime', 'Drama'], ['Biography', 'Drama', 'History'], ['Action', 'Adventure', 'Drama', 'Fantasy'], ['Crime', 'Drama'], ['Western'], ['Action', 'Adventure', 'Drama', 'Fantasy'], ['Drama'], ['Drama', 'Romance'], ['Action', 'Adventure', 'Sci-Fi', 'Thriller'], ['Action', 'Adventure', 'Drama', 'Fantasy'], ['Action', 'Adventure', 'Fantasy', 'Sci-Fi'], ['Action', 'Sci-Fi'], ['Biography', 'Crime', 'Drama'], ['Drama'], ['Action', 'Adventure', 'Drama'], ['Crime', 'Drama', 'Mystery', 'Thriller'], ['Comedy', 'Drama', 'Romance', 'War'], ['Crime', 'Drama'], ['Crime', 'Drama', 'Thriller'], ['Drama', 'Family', 'Fantasy'], ['Action', 'Adventure', 'Fantasy', 'Sci-Fi'], ['Drama', 'War'], ['Animation', 'Adventure', 'Family', 'Fantasy', 'Mystery'], ['Crime', 'Drama', 'Fantasy', 'Mystery'], ['Adventure', 'Drama', 'Sci-Fi'], ['Comedy', 'Drama', 'Thriller'], ['Action', 'Crime', 'Drama', 'Thriller'], [

### STEP 3. CREATING A DICTIONARY

In [7]:
movies_genre_imdb ={}
keys = movie_names
count = 0
for i in keys:
    movies_genre_imdb[i] = genre_list[count]
    count = count + 1
    
print(movies_genre_imdb)

{'The Shawshank Redemption': ['Drama'], 'The Godfather': ['Crime', 'Drama'], 'The Godfather: Part II': ['Crime', 'Drama'], 'The Dark Knight': ['Action', 'Crime', 'Drama', 'Thriller'], '12 Angry Men': ['Crime', 'Drama'], "Schindler's List": ['Biography', 'Drama', 'History'], 'The Lord of the Rings: The Return of the King': ['Action', 'Adventure', 'Drama', 'Fantasy'], 'Pulp Fiction': ['Crime', 'Drama'], 'The Good, the Bad and the Ugly': ['Western'], 'The Lord of the Rings: The Fellowship of the Ring': ['Action', 'Adventure', 'Drama', 'Fantasy'], 'Fight Club': ['Drama'], 'Forrest Gump': ['Drama', 'Romance'], 'Inception': ['Action', 'Adventure', 'Sci-Fi', 'Thriller'], 'The Lord of the Rings: The Two Towers': ['Action', 'Adventure', 'Drama', 'Fantasy'], 'Star Wars: Episode V - The Empire Strikes Back': ['Action', 'Adventure', 'Fantasy', 'Sci-Fi'], 'The Matrix': ['Action', 'Sci-Fi'], 'Goodfellas': ['Biography', 'Crime', 'Drama'], "One Flew Over the Cuckoo's Nest": ['Drama'], 'Seven Samurai':

# WEBSITE: www.rottentomatoes.com

#### LINK: https://www.rottentomatoes.com/top/bestofrt/

### STEP 1. EXTRACTING MOVIE NAMES 

In [8]:
url = "https://www.rottentomatoes.com/top/bestofrt/"
page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.content, "html.parser")

container = soup.find('table', class_='table')
movie_names = container.find_all('a', class_ = 'unstyled articleLink')

import re
names = []
movie_names = movie_names[:50]
for i in range(len(movie_names)):
    temp = movie_names[i].text
    temp = re.sub(r'[^\w\s]','',temp)
    temp = re.sub('[0-9]+','',temp)
    temp = temp.strip()
    names.append(temp)
print(names)

['Black Panther', 'Avengers Endgame', 'Us', 'Toy Story', 'Lady Bird', 'Citizen Kane', 'Mission Impossible  Fallout', 'The Wizard of Oz', 'BlacKkKlansman', 'Get Out', 'The Irishman', 'Mad Max Fury Road', 'SpiderMan Into the SpiderVerse', 'Casablanca', 'Moonlight', 'A Star Is Born', 'Wonder Woman', 'Dunkirk', 'Inside Out', 'The Farewell', 'A Quiet Place', 'Modern Times', 'Eighth Grade', 'It Happened One Night', 'Booksmart', 'A Night at the Opera', 'Roma', 'The Third Man', 'Coco', 'Thor Ragnarok', 'The Shape of Water', 'Spotlight', 'Selma', 'The Godfather', 'La Grande illusion Grand Illusion', 'Arrival', 'Singin in the Rain', 'Snow White and the Seven Dwarfs', 'Logan', 'The Cabinet of Dr Caligari Das Cabinet des Dr Caligari', 'Double Indemnity', 'The Favourite', 'The Big Sick', 'SpiderMan Far From Home', 'ET The ExtraTerrestrial', 'Star Wars The Last Jedi', 'All About Eve', 'Shadow of a Doubt', 'The Kid', 'Paddington']


### STEP 2. EXTRACTING GENRES

In [9]:
#class meta-value genre

movie_links = []
movies = movie_names[:50]
for movie in movies:
    link = movie.get('href', None)
    link = "https://www.rottentomatoes.com"+link
    movie_links.append(link)
#print(movie_links)
#for i in range(len(movie_links)):
#    print(movie_links[i])
#len(movie_names)

In [10]:
glist=[]
arr = []
import re
for i in range(len(movie_links)):
    #print("Movie no:", i+1)
    page = requests.get(movie_links[i], headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")    
    tags = soup.find_all('div', class_ = 'meta-value genre') #find_all to return a list
    for each in tags:
        temp =each.text.strip()
        temp = temp.split(",");
        for t in temp:
            t = t.strip()
            arr.append(t)
        glist.append(arr[:])
        arr.clear()
print(glist)

[['fantasy', 'action', 'adventure'], ['fantasy', 'adventure', 'sci fi', 'action'], ['horror', 'mystery and thriller'], ['fantasy', 'kids and family', 'animation', 'adventure', 'comedy'], ['drama', 'comedy'], ['drama'], ['adventure', 'mystery and thriller', 'action'], ['musical', 'kids and family', 'fantasy'], ['drama', 'comedy', 'crime'], ['horror', 'mystery and thriller', 'comedy'], ['drama', 'crime'], ['adventure', 'action'], ['fantasy', 'comedy', 'kids and family', 'animation', 'adventure', 'action'], ['drama'], ['drama'], ['drama', 'music', 'romance'], ['adventure', 'action', 'fantasy'], ['drama', 'war', 'history'], ['comedy', 'fantasy', 'kids and family', 'animation'], ['drama', 'comedy'], ['horror', 'mystery and thriller'], ['comedy'], ['drama', 'comedy'], ['romance'], ['comedy'], ['comedy'], ['drama'], ['mystery and thriller'], ['music', 'kids and family', 'animation', 'adventure', 'comedy'], ['fantasy', 'comedy', 'sci fi', 'adventure', 'action'], ['fantasy', 'romance'], ['drama

### STEP 3. CREATING A DICTIONARY

In [11]:
movies_genre_rt ={}
keys = names
count = 0
for i in keys:
    movies_genre_rt[i] = glist[count]
    count = count + 1
    
print(movies_genre_rt)

{'Black Panther': ['fantasy', 'action', 'adventure'], 'Avengers Endgame': ['fantasy', 'adventure', 'sci fi', 'action'], 'Us': ['horror', 'mystery and thriller'], 'Toy Story': ['fantasy', 'kids and family', 'animation', 'adventure', 'comedy'], 'Lady Bird': ['drama', 'comedy'], 'Citizen Kane': ['drama'], 'Mission Impossible  Fallout': ['adventure', 'mystery and thriller', 'action'], 'The Wizard of Oz': ['musical', 'kids and family', 'fantasy'], 'BlacKkKlansman': ['drama', 'comedy', 'crime'], 'Get Out': ['horror', 'mystery and thriller', 'comedy'], 'The Irishman': ['drama', 'crime'], 'Mad Max Fury Road': ['adventure', 'action'], 'SpiderMan Into the SpiderVerse': ['fantasy', 'comedy', 'kids and family', 'animation', 'adventure', 'action'], 'Casablanca': ['drama'], 'Moonlight': ['drama'], 'A Star Is Born': ['drama', 'music', 'romance'], 'Wonder Woman': ['adventure', 'action', 'fantasy'], 'Dunkirk': ['drama', 'war', 'history'], 'Inside Out': ['comedy', 'fantasy', 'kids and family', 'animatio

### For now, we have 3 dictionaries,  from 3 websites.

#### 1. movies_genre_meta
#### 2. movies_genre_imdb
#### 3. movies_genre_rt

### Next we will be creating CSV files and DB files, clean the data and perform the analysis


## CREATING .CSV FILES

In [12]:
import os
import csv

list1=[]
for key in movies_genre_rt:
    temp=[]
    temp.append(key)
    for val in movies_genre_rt[key]:
        val = val.lower()
        val =re.sub(r'[^\w\s]','',val)
        temp.append(val)
    list1.append(temp)
#print(list1)

f = open('genre.csv', 'w')
with f:   
    writer = csv.writer(f,lineterminator='\n')
    for row in list1:
        writer.writerow(row)

list2=[]
for key in movies_genre_imdb:
    temp=[]
    temp.append(key)
    for val in movies_genre_imdb[key]:
        val = val.lower()
        temp.append(val)
    list2.append(temp)

f = open('genre.csv', 'a')
with f:   
    writer = csv.writer(f,lineterminator='\n')
    for row in list2:
        writer.writerow(row)
            
list3=[]
for key in movies_genre_meta:
    temp=[]
    temp.append(key)
    for val in movies_genre_meta[key]:
        val = val.lower()
        temp.append(val)
    list3.append(temp)

f = open('genre.csv', 'a')

with f:   
    writer = csv.writer(f,lineterminator='\n')
    for row in list3:
        writer.writerow(row)
        
        
print("CSV FILE CREATED.")

CSV FILE CREATED.


# CREATING THE KEYWORD FREQUENCY DICTIONARY

In [13]:
imdbDict={}

for key in movies_genre_imdb:
    for val in movies_genre_imdb[key]:
        val=val.lower()
        val = re.sub(r'[^\w\s]','',val)
        val = val.replace(" ","")
        if val in imdbDict:
            imdbDict[val]+=1
        else:
            imdbDict[val]=1
rtDict={}

for key in movies_genre_rt:
    for val in movies_genre_rt[key]:
        val=val.lower()
        val = re.sub(r'[^\w\s]','',val)
        val = val.replace(" ","")
        if val in rtDict:
            rtDict[val]+=1
        else:
            rtDict[val]=1
            
metaDict={}

for key in movies_genre_meta:
    for val in movies_genre_meta[key]:
        val=val.lower()
        val = re.sub(r'[^\w\s]','',val)
        val = val.replace(" ","")
        if val in metaDict:
            metaDict[val]+=1
        else:
            metaDict[val]=1
# print("Imdb-")
# print(imdbDict)

# print("\n")
# print("Rottentomatoes-")
# print(rtDict)
# print("\n")
# print("Metacritics")
# print(metaDict)


for key in imdbDict:
    if key not in rtDict:
        rtDict[key] = 0
    if key not in metaDict:
        metaDict[key] = 0
        

for key in rtDict:
    if key not in imdbDict:
        imdbDict[key] = 0
    if key not in metaDict:
        metaDict[key] = 0
        
for key in metaDict:
    if key not in rtDict:
        rtDict[key] = 0
    if key not in imdbDict:
        imdbDict[key] = 0
        
    
print("Imdb-")    
print(imdbDict)

print('\n')

print("Rotten Tomatoes-")
print(rtDict)
print('\n')

print("Metacritics-")
print(metaDict)
print('\n')

Imdb-
{'drama': 39, 'crime': 13, 'action': 13, 'thriller': 10, 'biography': 5, 'history': 2, 'adventure': 12, 'fantasy': 8, 'western': 2, 'romance': 5, 'scifi': 8, 'mystery': 7, 'comedy': 6, 'war': 5, 'family': 4, 'animation': 3, 'musical': 2, 'music': 2, 'horror': 1, 'mysteryandthriller': 0, 'kidsandfamily': 0, 'filmnoir': 0, 'sport': 0, 'documentary': 0}


Rotten Tomatoes-
{'fantasy': 13, 'action': 10, 'adventure': 14, 'scifi': 5, 'horror': 4, 'mysteryandthriller': 7, 'kidsandfamily': 8, 'animation': 5, 'comedy': 18, 'drama': 21, 'musical': 2, 'crime': 5, 'music': 2, 'romance': 4, 'war': 2, 'history': 2, 'thriller': 0, 'biography': 0, 'western': 0, 'mystery': 0, 'family': 0, 'filmnoir': 0, 'sport': 0, 'documentary': 0}


Metacritics-
{'drama': 39, 'mystery': 10, 'thriller': 13, 'crime': 4, 'romance': 11, 'war': 5, 'filmnoir': 4, 'comedy': 9, 'musical': 3, 'history': 5, 'fantasy': 3, 'animation': 3, 'family': 3, 'adventure': 3, 'western': 2, 'action': 3, 'sport': 1, 'documentary': 1, 

# Calculating Scores:

<img src="image.png">

# Score between IMDB & RottenTomatoes

In [14]:
#SCORE BETWEEN IMDB & RT
import math

line1=[]
line2=[]

for key in imdbDict:
    line1.append(imdbDict[key])
    line2.append(rtDict[key])
        
# print(line1)
# print(line2)
        
# print(imdbDict)
# print(rtDict)        
        
        
# print("line 1 len:",len(line1))
# print(line1)
# print("line 2 len:",len(line2))
# print(line2)

num=0
for i in range(len(line1)):
    num+=line1[i]*line2[i]


    
den1=0
for i in range(len(line1)):
    den1+=line1[i]**2
den1=math.sqrt(den1)
den2=0


for i in range(len(line1)):
    den2+=line2[i]**2
den2=math.sqrt(den2)

den=den1*den2
# print(num)

print(num/den)

0.7915952017022123


# Score between IMDB & MetaCritics

In [15]:
#SCORE BETWEEN IMDB & META
import math

line1=[]
line2=[]

for key in imdbDict:
    line1.append(imdbDict[key])
    line2.append(metaDict[key])


# print("line 1 len:",len(line1))
# print(line1)
# print("line 2 len:",len(line2))
# print(line2)

num=0

for i in range(len(line1)):
    num+=line1[i]*line2[i]

den1=0

for i in range(len(line1)):
    den1+=line1[i]**2
den1=math.sqrt(den1)
den2=0
for i in range(len(line1)):
    den2+=line2[i]**2
den2=math.sqrt(den2)

den=den1*den2
# print(num)

print(num/den)

0.9019873939979214


# Score between RottenTomatoes & MetaCritics

In [16]:
#SCORE BETWEEN RT & META
import math

line1=[]
line2=[]

for key in metaDict:
    line1.append(rtDict[key])
    line2.append(metaDict[key])
       
# print("line 1 len:",len(line1))
# print(line1)
# print("line 2 len:",len(line2))
# print(line2)


num=0

for i in range(len(line1)):
    num+=line1[i]*line2[i]

den1=0

for i in range(len(line1)):
    den1+=line1[i]**2
den1=math.sqrt(den1)
den2=0
for i in range(len(line1)):
    den2+=line2[i]**2
den2=math.sqrt(den2)

den=den1*den2
# print(num)

print(num/den)

0.6824329998331107
